In [226]:
import json
import pandas

In [227]:
FILENAME = '../json/info.json'

with open(FILENAME, encoding='utf8') as info:
    INFO = json.load(info)

In [228]:
RANK1 = ['r2005', 'r2006#1', 'r2006#2']
RANK2 = ['r2007', 'r2008', 'r2009', 'r2010', 'r2011', 'r2012', 'r2013', 'r2014', 'r2015', 'r2016', 'r2017', 'r2018', 'r2019']
RANK = RANK1 + RANK2
URA = ['r2019u']
RECENT = 'r2019'

def year_filter(value: str, initial: str, array: list, mode: str = 'ge'):
    if mode not in ['gt', 'ge']:
        raise ValueError(f'invalid mode: {mode}')
    if value in array and initial in array:
        if mode == 'ge':
            return array.index(value) >= array.index(initial)
        else:
            return array.index(value) > array.index(initial)
    elif value in array: # 初登場が2006以前
        return True
    else:
        return False

In [229]:
# DataFrame 作成用のサマリJSONを作り直す

# keys = [
#     'no', # index
#     'title', # 曲名
#     'summation', # オモテ合計順位
#     'appearance', # オモテ圏内登場回数
#     'inst', # インスト
#     'hall_of_fame', # 殿堂入り
#     'works', # 収録作品
#     'debut', # 初出作品
#     'date', # 発売日
#     'initial', # 初登場ランキング
#     'kind', # 収録作品種別
#     'r2005', 'r2006#1', 'r2006#2',
#     'r2007', 'r2008', 'r2009', 'r2010', 'r2011', 'r2012', 'r2013', 'r2014', 'r2015', 'r2016', 'r2017', 'r2018', 'r2019',
#     'r2019u'
# ]

# SONGS = INFO['songs']
# WORKS = INFO['works']
# MAX = INFO['max']
# LENGTH = len(SONGS)

# SUMMARY = {key: [None] * LENGTH for key in keys}

# for i in range(LENGTH):
#     SONG = SONGS[f'{i + 1:03d}']
#     SUMMARY['no'][i] = i + 1
#     SUMMARY['title'][i] = SONG['title']
#     SUMMARY['summation'][i] = 0
#     SUMMARY['appearance'][i] = 0
#     SUMMARY['inst'][i] = SONG['inst']
#     SUMMARY['hall_of_fame'][i] = SONG['hall_of_fame']
#     SUMMARY['works'][i] = [WORKS[work[:3]]['title'] for work in SONG['works']]
#     SUMMARY['debut'][i] = WORKS[SONG['debut']]['title']
#     SUMMARY['date'][i] = WORKS[SONG['debut']]['release']
#     SUMMARY['initial'][i] = SONG['initial']
#     SUMMARY['kind'][i] = SONG['kind']

#     for r in RANK:
#         if r in SONG:
#             SUMMARY[r][i] = SONG[r] if SONG[r] is not None else MAX[r]
#             if r in RANK2:
#                 SUMMARY['summation'][i] += SUMMARY[r][i]
#                 SUMMARY['appearance'][i] += 1

#     for r in URA:
#         if r in SONG:
#             SUMMARY[r][i] = int(SONG[r])

# del SONGS, WORKS, MAX, LENGTH

# with open('../data/summary.json', 'w', encoding='utf8') as summary:
#     json.dump(SUMMARY, summary, ensure_ascii=False)

SUMMARY = pandas.read_json('../data/summary.json')
SUMMARY.index = SUMMARY.no
SUMMARY = SUMMARY.drop(['no'], 1)
SUMMARY['average'] = SUMMARY.summation / SUMMARY.appearance
SUMMARY.date = pandas.to_datetime(SUMMARY.date)

In [230]:
# 今回の対象楽曲 (100位以下、殿堂入り、インストを除外)
TARGET = SUMMARY.query('not hall_of_fame and not inst and r2019 > 100')

TARGET['over100'] = 0
TARGET['count_all'] = 0
TARGET['average_exclude_debut'] = None
TARGET['highest_exclude_debut'] = None

for no in TARGET.index:
    song = TARGET.loc[no]
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK), RANK))
    TARGET.over100.at[no] = len(list(filter(lambda s: s > 100, song[to_mean_all].values)))
    TARGET.count_all.at[no] = len(to_mean_all)
    to_mean_exclude_debut = list(filter(lambda y: year_filter(y, song.initial, RANK2, 'gt'), RANK2))
    TARGET.average_exclude_debut.at[no] = song[to_mean_exclude_debut].mean()
    TARGET.highest_exclude_debut.at[no] = song[to_mean_exclude_debut].min()

TARGET['rank_average_exclude_debut'] = TARGET.average_exclude_debut.rank(method='min', na_option='bottom')
TARGET['rank_highest_exclude_debut'] = TARGET.highest_exclude_debut.rank(method='min', na_option='bottom')

In [231]:
TARGET

,title,summation,appearance,inst,hall_of_fame,works,debut,date,initial,kind,...,r2018,r2019,r2019u,average,over100,count_all,average_exclude_debut,highest_exclude_debut,rank_average_exclude_debut,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1579,13,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,r2005,"[SA, OA, OT]",...,164.0,123.0,64.0,121.461538,9,16,121.462,68,46.0,64.0
3,マリーゴールド・ガーデン,2233,13,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,r2005,"[SA, OA, S-2]",...,164.0,216.0,128.0,171.769231,13,16,171.769,141,120.0,164.0
4,鏡面界 im Juni,2614,13,False,False,[幻想庭園],幻想庭園,1988-01-25,r2005,[OA],...,250.0,260.0,150.0,201.076923,13,16,201.077,131,162.0,152.0
5,アンジェ・ノワールの祭戯,1633,13,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,r2005,"[SA, OA]",...,63.0,106.0,NaN,125.615385,9,16,125.615,63,52.0,55.0
6,紅い睡蓮の午後,2454,13,False,False,"[幻想庭園, 月光嗜好症]",幻想庭園,1988-01-25,r2005,"[SA, OA]",...,209.0,226.0,152.0,188.769231,13,16,188.769,136,143.0,159.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,Taolor Tの変身譚,183,2,False,False,[芸術変態論],芸術変態論,2018-07-25,r2018,[OA],...,47.0,136.0,NaN,91.500000,1,2,136,136,64.0,159.0
328,ヤマトイズム,196,2,False,False,[芸術変態論],芸術変態論,2018-07-25,r2018,[OA],...,26.0,170.0,NaN,98.000000,1,2,170,170,114.0,190.0
330,眼球ロマン,277,2,False,False,[芸術変態論],芸術変態論,2018-07-25,r2018,[OA],...,107.0,170.0,67.0,138.500000,2,2,170,170,114.0,190.0


## ウラ対象楽曲リスト

In [232]:
col = 4
length = len(TARGET)
row = int(pandas.np.ceil(length/2))

a2 = [[] for _ in range(col)]
c = 0
for no in TARGET.index:
    a2[int(c // row)] += [TARGET.title.at[no]]
    c += 1

# write to file (html tag)
with open('../data_ura/list.txt', 'w', encoding='utf8') as list_file:
    list_file.write('<div style="display: flex; justify-content: space-around; border: solid 1px; overflow: auto; flex-wrap: wrap;">\n')

    for i in range(2):
        j = 0
        list_file.write(f'<ol start="{i * row + j + 1}" style="padding-left: 5px;">\n')
        for j in range(len(a2[i])):
            list_file.write(f'<li>{a2[i][j]}</li>')
        list_file.write('\n</ol>\n')
    list_file.write('</div>')

## 昨年度振り返り

In [233]:
# # 初登場を除く平均を追加
# TARGET['average_exclude_debut'] = 0
# # 最高順位を追加
# TARGET['highest'] = 0
# TARGET['lowest'] = 0
# # 初登場を除く最高順位を追加
# TARGET['highest_exclude_debut'] = 0


LAST = SUMMARY.query(f'r2018 >= 100')
RANK_LAST_YEAR = RANK2[:-1]
RANK_ALL_LAST_YEAR = RANK[:-1]

LAST['average_exclude_debut'] = None
LAST['highest_exclude_debut'] = None
# LAST['highest'] = 0
# LAST['lower'] = 0
# LAST['over30'] = 0
# LAST['over50'] = 0
LAST['over100'] = None
# LAST['over200'] = 0
LAST['count_all'] = None

for no in LAST.index:
    song = LAST.loc[no]
    to_mean = list(filter(lambda y: year_filter(y, song.initial, RANK_LAST_YEAR), RANK_LAST_YEAR))
    to_mean_exclude_debut = list(filter(lambda y: year_filter(y, song.initial, RANK_LAST_YEAR, 'gt'), RANK_LAST_YEAR))
    LAST.average.at[no] = song[to_mean].mean()
    LAST.average_exclude_debut.at[no] = song[to_mean_exclude_debut].mean()
    LAST.highest_exclude_debut.at[no] = song[to_mean_exclude_debut].min()
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL_LAST_YEAR), RANK_ALL_LAST_YEAR))
    # LAST.highest.at[no] = song[to_mean_all].min()
    # LAST.lowest.at[no] = song[to_mean_all].max()
    # LAST.over30.at[no] = len(list(filter(lambda s: s > 30, song[to_mean_all].values)))
    # LAST.over50.at[no] = len(list(filter(lambda s: s > 50, song[to_mean_all].values)))
    LAST.over100.at[no] = len(list(filter(lambda s: s > 100, song[to_mean_all].values)))
    # LAST.over200.at[no] = len(list(filter(lambda s: s > 200, song[to_mean_all].values)))
    LAST.count_all.at[no] = len(to_mean_all)
    LAST.at[no, 'count'] = len(to_mean)

LAST['under100'] = (LAST.count_all - LAST.over100) / LAST.count_all
LAST['rank_average'] = LAST.average_exclude_debut.rank(method='min', na_option='bottom')
LAST['rank_highest'] = LAST.highest_exclude_debut.rank(method='min', na_option='bottom')

tmp = LAST.sort_values(['r2019u']).query('r2019u <= 20')[['title','date','r2019u', 'r2019', 'under100', 'rank_average', 'average_exclude_debut', 'rank_highest', 'highest_exclude_debut']]

with open('../data_ura/review.md', 'w', encoding='utf8') as file:
    file.write('ウラ順位|曲名|発表年|2019本戦|100位圏内率|平均順位の順位|最高順位の順位\n')
    file.write('---:|---|---:|---:|---:|---:|---:\n')
    for no in tmp.index:
        row = tmp.loc[no]
        file.write(f'''{int(row.r2019u)}|{'<span style="color: red;">' if int(row.r2019) <= 100 else ''}{row.title}{'</span>' if int(row.r2019) <= 100 else ""}|{row.date.year}|{int(row.r2019)}|{row.under100:.3f}|{int(row.rank_average)}|{int(row.rank_highest)}\n''')

## 今回の対象楽曲のうち、去年のウラで高い順位だった曲

In [234]:
tmp = TARGET.sort_values(['r2019u']).query('r2019u <= 30')[['title', 'r2019u', RECENT, 'date']]

with open('../data_ura/top_in_last_ura.md', 'w', encoding='utf8') as file:
    file.write('ウラ順位|曲名|発表年|2019本戦\n')
    file.write('---:|---|---:|---:\n')
    for no in tmp.index:
        row = tmp.loc[no]
        file.write(f'{int(row.r2019u)}|{row.title}|{row.date.year}|{int(row.r2019)}\n')

## 初トップ100圏外

In [235]:
tmp = TARGET[(100 < TARGET[RECENT])].query('over100 == 1')[['title', 'date', *RANK2]]

with open('../data_ura/first100.md', 'w', encoding='utf8') as file:
    file.write('曲名|発表年|最低順位(2019 以前)|2019\n')
    file.write('---|---:|---:|---:\n')
    for no in tmp.index:
        row = tmp.loc[no]
        file.write(f'{row.title}|{row.date.year}|{int(row[RANK2[:-1]].max()) if row.date.year < 2019 else "-"}|{int(row.r2019)}\n')

## トップ100ランクイン割合が多い曲

In [236]:
top100 = TARGET.loc[:, :]
top100['under100'] = top100.count_all - top100.over100
top100['under100_rate'] = top100.under100 / top100.count_all
top100['rank_under100_rate'] = top100.under100_rate.rank(method='min', na_option='bottom', ascending=False)
tmp = top100.sort_values(['under100_rate', 'under100'], ascending=False) \
   .query('under100_rate > 0.7') \
   [['title'] + RANK2 + ['under100',  'under100_rate', 'rank_under100_rate', 'date']]

with open('../data_ura/top100.txt', 'w', encoding='utf8') as file:
   file.write('<table>\n')
   file.write('<tr><th>順位</th><th>曲名</th><th>発表年</th><th>回数</th><th>率</th></tr>\n')
   before = 0
   for no in tmp.index:
      row = tmp.loc[no]
      file.write('<tr>')
      if before != row.rank_under100_rate:
         count = sum(tmp.rank_under100_rate == row.rank_under100_rate)
         file.write(f'''<td{'' if count < 2 else f' rowspan="{count}"'}>{int(row.rank_under100_rate)}</td>''')
      file.write(f'<td>{row.title}</td><td>{row.date.year}</td><td>{row.under100}</td><td>{row.under100_rate:.3f}</td></tr>\n')
      before = row.rank_under100_rate
   file.write('</table>')

## 過去平均順位が高い曲

In [239]:
average_top = TARGET.sort_values(['rank_average_exclude_debut', 'date'], ascending=[True, True]) \
                    .query('average_exclude_debut <= 100') \
                    [['title', 'rank_average_exclude_debut', 'average_exclude_debut', 'date']].astype({'rank_average_exclude_debut': int})

with open('../data_ura/average_top.md', 'w', encoding='utf8') as file:
    file.write('\#|曲名|平均\n')
    file.write('---:|---|---:\n')
    for no in average_top.index:
        file.write(f'{average_top.rank_average_exclude_debut.at[no]}|{average_top.title.at[no]}|{average_top.average_exclude_debut.at[no]:.2f}\n')

,title,rank_average_exclude_debut,average_exclude_debut,date
no,,,,
185,神風,1,39.4,2008-08-27
261,真夏の憂愁夫人,2,44.8571,2012-07-18
269,ローズ家の双子達,3,52.3333,2013-09-11
268,令嬢薔薇図鑑,4,59.1667,2013-09-11
130,昭和恋々幻燈館,5,62.6923,2005-06-22
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,6,63.2222,2009-08-26
195,鬼帝の剣,7,64.3,2008-11-19
264,快恠奇奇,8,66.5,2013-02-20
181,コトダマ,9,67,2008-01-23


## 過去平均最高順位が高い曲

In [241]:
highest_top = TARGET.sort_values(['rank_highest_exclude_debut', 'date'], ascending=[True, True]) \
                    .query('highest_exclude_debut <= 50') \
                    [['title', 'rank_highest_exclude_debut', 'highest_exclude_debut', 'date']].astype({'rank_highest_exclude_debut': int, 'highest_exclude_debut': int})

with open('../data_ura/highest.md', 'w', encoding='utf8') as file:
    file.write('\n')

highest_top

,title,rank_highest_exclude_debut,highest_exclude_debut,date
no,,,,
261,真夏の憂愁夫人,1,9,2012-07-18
274,黒百合隠密カゲキダン,2,10,2013-09-11
269,ローズ家の双子達,3,12,2013-09-11
185,神風,4,14,2008-08-27
215,堕天國宣戦,4,14,2009-10-21
206,お毒味LADY,6,18,2009-08-26
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,7,19,2009-08-26
195,鬼帝の剣,8,23,2008-11-19
268,令嬢薔薇図鑑,9,25,2013-09-11


## 未投票楽曲

In [243]:
# 未投票
zero = ['日曜日のシエスタ', 'La vie en rose', 'Féliciter', 'Nocturnal', '寶島', '逢魔蛾城の伯爵', '小夜すみれ ～ある夜の森の小さなお話', '魂ノ代']
TARGET.query('title in @zero')

,title,summation,appearance,inst,hall_of_fame,works,debut,date,initial,kind,...,average,over100,count_all,average_exclude_debut,highest_exclude_debut,rank_average_exclude_debut,rank_highest_exclude_debut,under100,under100_rate,rank_under100_rate
no,,,,,,,,,,,,,,,,,,,,,
55,日曜日のシエスタ,2476,13,False,False,"[CLAMP学園探偵団 Vocal Collection, étoiles]",CLAMP学園探偵団 Vocal Collection,1997-12-17,r2005,"[SA, ST]",...,190.461538,13,16,190.462,129,146.0,148.0,3,0.1875,107.0
91,La vie en rose,2812,13,False,False,[CLASSICS],CLASSICS,2001-07-25,r2005,[OA],...,216.307692,13,16,216.308,155,192.0,178.0,3,0.1875,107.0
106,Féliciter,2663,13,False,False,[月光嗜好症],月光嗜好症,2003-04-23,r2005,[SA],...,204.846154,13,16,204.846,168,170.0,188.0,3,0.1875,107.0
180,Nocturnal,2673,12,False,False,[Grand Finale],Grand Finale,2007-12-12,r2008,[SA],...,222.750000,12,12,229,192,204.0,204.0,0,0.0000,179.0
216,寶島,2090,10,False,False,"[堕天國宣戦, QUEENDOM]",堕天國宣戦,2009-10-21,r2010,"[S-2, CA]",...,209.000000,10,10,220.111,147,198.0,170.0,0,0.0000,179.0
247,逢魔蛾城の伯爵,1730,9,False,False,[Les Papillons],Les Papillons,2011-06-29,r2011,[SA],...,192.222222,9,9,200.875,142,161.0,165.0,0,0.0000,179.0
296,小夜すみれ ～ある夜の森の小さなお話,1058,5,False,False,[Violetta Operetta],Violetta Operetta,2015-01-21,r2015,[SA],...,211.600000,5,5,215.75,153,191.0,174.0,0,0.0000,179.0
311,魂ノ代,1162,5,False,False,[波羅蜜蓮華],波羅蜜蓮華,2015-10-21,r2015,[S-2],...,232.400000,5,5,247.75,237,210.0,210.0,0,0.0000,179.0


## シングル表題

In [244]:
TARGET[TARGET.kind.apply(lambda k: 'S-1' in k)]

,title,summation,appearance,inst,hall_of_fame,works,debut,date,initial,kind,...,average,over100,count_all,average_exclude_debut,highest_exclude_debut,rank_average_exclude_debut,rank_highest_exclude_debut,under100,under100_rate,rank_under100_rate
no,,,,,,,,,,,,,,,,,,,,,
12,フラワーチャイルド,2698,13,False,False,"[フラワーチャイルド, 幻想庭園, 快恠奇奇 ALI PROJECT Ventennale ...",フラワーチャイルド,1988-07-25,r2005,"[OA, S-1, CA]",...,207.538462,13,16,207.538,133,175.0,153.0,3,0.187500,107.0
13,恋せよ乙女 -Love story of ZIPANG-,1194,13,False,False,"[恋せよ乙女 -Love story of ZIPANG-, 月下の一群, jamais v...",恋せよ乙女 -Love story of ZIPANG-,1992-07-07,r2005,"[OA, S-1, SA, CA]",...,91.846154,4,16,91.8462,36,19.0,19.0,12,0.750000,11.0
43,Wish,1537,13,False,False,"[Wish, music tracks from Wish, CLAMPAZAR ～クランパ...",Wish,1996-12-24,r2005,"[S-1, OT, ST, CA]",...,118.230769,7,16,118.231,48,42.0,33.0,9,0.562500,35.0
57,Anniversary of Angel,1797,13,False,False,"[Anniversary of Angel, CLAMPAZAR ～クランパザール～, CO...",Anniversary of Angel,1997-12-17,r2005,"[S-1, OT, CA]",...,138.230769,8,16,138.231,54,66.0,41.0,8,0.500000,39.0
59,LABYRINTH,1554,13,False,False,"[LABYRINTH, Noblerot, 聖ルミナス女学院2 O.S.T, 快恠奇奇 AL...",LABYRINTH,1998-10-21,r2005,"[OA, S-1, ST, CA]",...,119.538462,9,16,119.538,59,45.0,47.0,7,0.437500,53.0
120,pastel pure,2586,13,False,False,"[マリア様がみてる オリジナルサウンドトラック, pastel pure, マリア様がみてる...",pastel pure,2004-08-25,r2005,"[S-1, ST, CA]",...,198.923077,13,16,198.923,104,160.0,116.0,3,0.187500,107.0
181,コトダマ,745,12,False,False,"[コトダマ, シゴフミ オリジナルサウンドトラック, La Vita Romantica, ...",コトダマ,2008-01-23,r2008,"[S-1, ST, CA]",...,62.083333,1,12,67,32,9.0,14.0,11,0.916667,2.0
195,鬼帝の剣,648,11,False,False,"[鬼帝の剣, 桂冠詩人 SINGLE COLLECTION PLUS, 鉄のラインバレル オ...",鬼帝の剣,2008-11-19,r2009,"[S-1, ST, CA]",...,58.909091,2,11,64.3,23,7.0,8.0,9,0.818182,8.0
198,裸々イヴ新世紀,1020,11,False,False,"[裸々イヴ新世紀, 宇宙をかける少女 オリジナルサウンドトラック, COMPILATION ...",裸々イヴ新世紀,2009-01-21,r2009,"[S-1, OT, ST, CA]",...,92.727273,4,11,101.1,29,24.0,12.0,7,0.636364,25.0


## 去年投票した曲

In [245]:
my_before = ['桜の花は狂い咲き', '乙女の祈り', 'Rose Moon', '少女貴族', '熱帯性植物園', '遊月恋歌', '少女殉血', 'Animals on the Earth', '四神獣飼殺し', 'ALICE同罪イノセント', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
TARGET.query('title in @my_before')

,title,summation,appearance,inst,hall_of_fame,works,debut,date,initial,kind,...,average,over100,count_all,average_exclude_debut,highest_exclude_debut,rank_average_exclude_debut,rank_highest_exclude_debut,under100,under100_rate,rank_under100_rate
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1579,13,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,r2005,"[SA, OA, OT]",...,121.461538,9,16,121.462,68,46.0,64.0,7,0.437500,53.0
209,Animals on the Earth,1145,10,False,False,[Poison],Poison,2009-08-26,r2010,[OA],...,114.500000,5,10,123.333,45,49.0,27.0,5,0.500000,39.0
234,四神獣飼殺し,1110,9,False,False,[汎新日本主義],汎新日本主義,2010-09-26,r2011,[OA],...,123.333333,5,9,135.375,71,63.0,74.0,4,0.444444,50.0
299,陸と海と空と,375,5,False,False,"[快楽のススメ, 愛と誠 ～YAMATO＆LOVE×××]",快楽のススメ,2015-09-09,r2015,"[OA, CA]",...,75.000000,2,5,89.25,56,17.0,43.0,3,0.600000,29.0
309,緋ノ糸輪廻ノGEMINI,523,5,False,False,[快楽のススメ],快楽のススメ,2015-09-09,r2015,[OA],...,104.600000,2,5,122.5,80,47.0,85.0,3,0.600000,29.0


## 今年投票した曲

In [247]:
my = ['桜の花は狂い咲き', "Nous Deux C'est Pour La Vie", '月夜のピエレット', '神風', 'Animals on the Earth', '絶國TEMPEST', '四神獣飼殺し', 'La vérité', '真夏の憂愁夫人', '令嬢薔薇図鑑', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
TARGET.query('title in @my')

,title,summation,appearance,inst,hall_of_fame,works,debut,date,initial,kind,...,average,over100,count_all,average_exclude_debut,highest_exclude_debut,rank_average_exclude_debut,rank_highest_exclude_debut,under100,under100_rate,rank_under100_rate
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1579,13,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,r2005,"[SA, OA, OT]",...,121.461538,9,16,121.462,68,46.0,64.0,7,0.437500,53.0
37,Nous Deux C'est Pour La Vie,1595,13,False,False,"[雨のソナタ ～La Pluie～, jamais vu, Déjà Vu ～THE ORI...",雨のソナタ ～La Pluie～,1995-10-18,r2005,"[S-2, CA]",...,122.692308,9,16,122.692,44,48.0,26.0,7,0.437500,53.0
49,月夜のピエレット,1811,13,False,False,"[ピアニィ・ピンク, CLAMP学園探偵団 O.S.T 1, CLAMP学園探偵団 Voca...",ピアニィ・ピンク,1997-05-21,r2005,"[OA, OT, S-2, ST, CA]",...,139.307692,10,16,139.308,68,68.0,64.0,6,0.375000,67.0
185,神風,407,11,False,False,"[禁書, 愛と誠 ～YAMATO＆LOVE×××]",禁書,2008-08-27,r2009,"[OA, CA]",...,37.000000,1,11,39.4,14,1.0,4.0,10,0.909091,3.0
209,Animals on the Earth,1145,10,False,False,[Poison],Poison,2009-08-26,r2010,[OA],...,114.500000,5,10,123.333,45,49.0,27.0,5,0.500000,39.0
228,絶國TEMPEST,1447,9,False,False,"[汎新日本主義, 愛と誠 ～YAMATO＆LOVE×××]",汎新日本主義,2010-09-26,r2011,"[OA, CA]",...,160.777778,6,9,174.25,69,123.0,70.0,3,0.333333,75.0
234,四神獣飼殺し,1110,9,False,False,[汎新日本主義],汎新日本主義,2010-09-26,r2011,[OA],...,123.333333,5,9,135.375,71,63.0,74.0,4,0.444444,50.0
258,La vérité,736,8,False,False,[贋作師],贋作師,2012-07-18,r2012,[OA],...,92.000000,3,8,103,60,27.0,48.0,5,0.625000,27.0
261,真夏の憂愁夫人,353,8,False,False,"[贋作師, 愛と誠 ～YAMATO＆LOVE×××]",贋作師,2012-07-18,r2012,"[OA, CA]",...,44.125000,1,8,44.8571,9,2.0,1.0,7,0.875000,4.0
